In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from dotenv import load_dotenv 
load_dotenv()
import yaml
import os
from crewai import Agent, Task, Crew


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\achafi\Documents\PProjects\AI_agents\MyPortfolioChatbot\.venv\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\achafi\Documents\PProjects\AI_agents\MyPortfolioChatbot\.venv\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\achafi\Documents\PProjects\AI_agents\MyPortfolioChatbot\.

AttributeError: _ARRAY_API not found

SystemError: <built-in function __import__> returned a result with an exception set

In [2]:
# Define file paths for YAML configurations
files = {
    'agents': r'src\portfolio_assistant\config\agents.yaml',
    'tasks': r'src\portfolio_assistant\config\tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

In [3]:
agents_config

{'resume_assistant': {'role': 'Resume assistant\n',
  'goal': 'Review the resume CV_Assia_Chafi_2025.pdf and extract key details about experience, skills, projects, and achievements.\n',
  'backstory': "You're a knowledgeable and friendly assistant trained on Assia Chafi's resume. Your job is to provide clear and helpful answers to Assia's portfolio website visitors. You are known for your ability to find the most relevant information and present it in a clear and concise manner.\n"},
 'reporting_analyst': {'role': 'Reporting Analyst\n',
  'goal': 'Create a brief based on your input\n',
  'backstory': "You're a meticulous analyst with a keen eye for detail. You're known for your clear and concise reports, making it easy for others to understand the information you provide."}}

In [4]:
tasks_config

{'extract_cv_info': {'description': 'Review the resume CV_Assia_Chafi_2025.pdf and extract key details about experience, skills, projects, and achievements.\n',
  'expected_output': 'JSON response following the `CVInfo` model with: - Work experience (list of job roles, companies, dates, descriptions) - Education (list of degrees, institutions, graduation years) - Skills (list of skills) - Projects (list of project names, descriptions, and tech used) - Certifications (list of certifications and issuing organizations)\n',
  'agent': 'resume_assistant'},
 'reporting_task': {'description': 'Respond to visitors {question} about Assia Chafi skills, experience, and portfolio using the extracted resume information.\n',
  'expected_output': 'JSON response following the `VisitorResponse` model with: - {question} - A clear answer using `CVInfo` - References to relevant CV sections for transparency.\n',
  'agent': 'reporting_analyst'}}

In [5]:
from pydantic import BaseModel
from typing import List

class WorkExperience(BaseModel):
    job_title: str
    company: str
    start_date: str
    end_date: str
    description: str


class Education(BaseModel):
    degree: str
    institution: str
    graduation_year: str


class Project(BaseModel):
    name: str
    description: str
    technologies: List[str]


class Certification(BaseModel):
    name: str
    issuing_organization: str


class CVInfo(BaseModel):
    work_experience: List[WorkExperience]
    education: List[Education]
    skills: List[str]
    projects: List[Project]
    certifications: List[Certification]


class CVReference(BaseModel):
    section: str  # Example: "work_experience", "skills", "education"
    details: str  # Summary of relevant data from `CVInfo`


class VisitorResponse(BaseModel):
    question: str
    answer: str
    references: List[CVReference]  # Where the answer is derived from

In [11]:
from crewai_tools import PDFSearchTool
pdf_search_tool = PDFSearchTool(pdf="knowledge/CV_Assia_Chafi_2025.pdf")
pdf_search_tool.description = "Search the CV PDF with a text query to find relevant information"
# Now create your agents
from crewai import Agent, Task
researcher = Agent(config=agents_config["resume_assistant"], verbose=True)
reporting_analyst = Agent(config=agents_config["reporting_analyst"], verbose=True)

research_task = Task(
    description=tasks_config["extract_cv_info"]["description"],
    expected_output=tasks_config["extract_cv_info"]['expected_output'],
    output_pydantic=CVInfo,
    tools=[pdf_search_tool],
    agent = researcher
)

reporting_task = Task(
    description=tasks_config["reporting_task"]['description'],
    expected_output=tasks_config["reporting_task"]['expected_output'],
    output_pydantic=VisitorResponse,
    output_file="report.md",
    agent=reporting_analyst
)

In [12]:
# Creating Crew
crew = Crew(
  agents=[
    researcher,
    reporting_analyst
  ],
  tasks=[
    research_task,
    reporting_task
  ],
  verbose=True
)

In [13]:
crew

Crew(id=0ffd448c-2e50-4fb3-943a-65544b33234e, process=Process.sequential, number_of_agents=2, number_of_tasks=2)

In [14]:
# The given Python dictionary
inputs = {
  'question': "What are your main skills?",
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 0ffd448c-2e50-4fb3-943a-65544b33234e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 804c2f8c-0576-4ec0-af8a-7cb5a85f64cf
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 804c2f8c-0576-4ec0-af8a-7cb5a85f64cf
       Status: Executing Task...
    └── 🤖 Agent: Resume assistant
        
            Status: In Progress

# Agent: Resume assistant
## Task: Review the resume CV_Assia_Chafi_2025.pdf and extract key details about experience, skills, projects, and achievements.



🚀 Crew: crew
└── 📋 Task: 804c2f8c-0576-4ec0-af8a-7cb5a85f64cf
       Status: Executing Task...
    └── 🤖 Agent: Resume assistant
        
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Reporting Analyst
## Final Answer: 
{
  "question": "What are your main skills?",
  "answer": "My main skills encompass a wide range of data science, machine learning, software engineering, and cloud computing capabilities.  I'm proficient in programming languages like Python (with libraries such as Pandas, NumPy, Scikit-learn, TensorFlow, Keras, and PyTorch), Java, and C++. My database skills include SQL, MySQL, PostgreSQL, and MongoDB. I'm also experienced with big data technologies like Spark and Hadoop, and cloud platforms such as AWS, Azure, and GCP.  I have a strong background in various machine learning techniques including regression, classification, clustering, deep learning, and time series analysis.  Furthermore, I possess skills in data visualization and reporting using tools like Tableau, Power BI, and Grafana, and am adept at ETL processes, dashboard creation, and KPI definition. My experience also extends to software development, including Android development 

🚀 Crew: crew
├── 📋 Task: 804c2f8c-0576-4ec0-af8a-7cb5a85f64cf
│      Assigned to: Resume assistant
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume assistant
│       
│           Status: ✅ Completed
│       └── 🔧 Used Search a PDF's content (3)
└── 📋 Task: 31924f88-f1fa-4ed7-bd5d-4eb7b8f360e5
       Status: Executing Task...
    └── 🤖 Agent: Reporting Analyst
        
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 804c2f8c-0576-4ec0-af8a-7cb5a85f64cf
│      Assigned to: Resume assistant
│   
│      Status: ✅ Completed
│   └── 🤖 Agent: Resume assistant
│       
│           Status: ✅ Completed
│       └── 🔧 Used Search a PDF's content (3)
└── 📋 Task: 31924f88-f1fa-4ed7-bd5d-4eb7b8f360e5
       Assigned to: Reporting Analyst
    
       Status: ✅ Completed
    └── 🤖 Agent: Reporting Analyst
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 31924f88-f1fa-4ed7-bd5d-4eb7b8f360e5                                                                     │
│  Agent: Reporting Analyst                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 0ffd448c-2e50-4fb3-943a-65544b33234e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result['answer']


"My main skills encompass a wide range of data science, machine learning, software engineering, and cloud computing capabilities.  I'm proficient in programming languages like Python (with libraries such as Pandas, NumPy, Scikit-learn, TensorFlow, Keras, and PyTorch), Java, and C++. My database skills include SQL, MySQL, PostgreSQL, and MongoDB. I'm also experienced with big data technologies like Spark and Hadoop, and cloud platforms such as AWS, Azure, and GCP.  I have a strong background in various machine learning techniques including regression, classification, clustering, deep learning, and time series analysis.  Furthermore, I possess skills in data visualization and reporting using tools like Tableau, Power BI, and Grafana, and am adept at ETL processes, dashboard creation, and KPI definition. My experience also extends to software development, including Android development (Java), Docker, and Kubernetes. I work effectively in Agile and Scrum environments."

: 